# Functions that were used

## Supplementary functions

In [ ]:
import pandas as pd
import os
import alignn
import numpy as np
from ase import Atoms
from ase.build import make_supercell, bulk
from ase.io import write
from itertools import combinations_with_replacement, product

def read_poscar(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    header = lines[:8] 
    atom_positions = lines[8:] 
    
    return header, atom_positions

def write_p(header, atom_positions, output_filename):
    with open(output_filename, 'w') as file:
        file.writelines(header)
        file.writelines(atom_positions)

def shuffle_poscar(input_file, n):
    header, atom_positions = read_poscar(input_file)
    unique_shuffles = set()

    i = 1
    while i < n:
        shuffled_positions = atom_positions.copy()
        np.random.shuffle(shuffled_positions) 
        
        shuffle_tuple = tuple(shuffled_positions)
        
        if shuffle_tuple not in unique_shuffles:  # Ensure uniqueness
            unique_shuffles.add(shuffle_tuple)
            output_file = f'{input_file[:-7]}-{i+1}.vasp'
            write_p(header, shuffled_positions, output_file)
            print(f"Generated shuffle: {output_file}")
            i += 1  # Increment only for unique shuffles

def calculate_combinations(atom_count, elements, minimum):
    """
    Creates different possible combinations of distributions of structure

    Args:
        atom_count (int): total number of atoms in structure
        elements (int): total number of elements in structure
        minimum (int): the minimum number of atoms an element should have
    
    Returns:
        Nested list of the different combinations of distributions of a structure
    """
    remaining = atom_count - elements * minimum
    equal = [minimum] * elements

    if remaining < 0:
        return []
    
    result = []
    for combination in product(range(remaining + 1), repeat=elements):
        if sum(combination) == remaining:
            result.append([minimum + combination[i] for i in range(elements)])
    
    return result

def compute_scaling(num_atoms, lattice_params):
    total_atoms = sum(num_atoms)
    atomic_fractions = [n / total_atoms for n in num_atoms]
    scaling_factor = sum(x * a for x, a in zip(atomic_fractions, lattice_params))
    
    return scaling_factor

from mp_api.client import MPRester


def calculate_lattice_a(atom_types):
    # Replace key-id-here with your respective key from the Materials Project API 
    with MPRester("key-id-here") as mpr:
        lattice_params = []
        for i, element in enumerate(atom_types):
            material = mpr.materials.summary.search(
                elements=[element], 
                fields=['energy_above_hull', 'material_id'])
            most_stable = min(material, key=lambda x: x.energy_above_hull)
            stable_material = mpr.materials.summary.search(
                material_ids=most_stable.material_id
            )
            primitive_structure = stable_material[0].structure
            conventional_structure = primitive_structure.to_conventional()
            lattice_constant = conventional_structure.lattice.a
            lattice_params.insert(i, lattice_constant)
    return lattice_params
    

## Main function to generate poscar files

In [ ]:
def gen_mass_poscar(atom_types:list, combinations:list, permutation:int, structure:str, folder: str):
    
    """
    Generates poscar files

    Args:
        atom_types (list): list of elements. Eg: ['Mo', 'V', 'Nb', 'Ti', 'Zr']
        combinations (list): list of combinations. Eg: [[1, 1, 1, 1, 1], [2, 2, 2, 2, 2]]
        permutation (int): number of different arrangements desired
        structure (str): 'bcc' or 'fcc'
        folder (str): directory for files to be stored in
    
    Returns:
        Poscar files with respect to parameters chosen
    """
    lattice_params = calculate_lattice_a(atom_types)
    os.makedirs(folder, exist_ok=True)
    
    if structure == 'bcc':
        unit_cell_atoms = 2
    elif structure == 'fcc':
        unit_cell_atoms = 4
    
    for i in combinations:
        # vegard's law
        scaling_factor = sum(i)/unit_cell_atoms
        s = round(scaling_factor ** (1/3))
        
        lattice_constant = compute_scaling(i, lattice_params)
        comment =  ''.join(str(num) for num in i)
        filename = f'{folder}/{comment}-1.vasp'

        # Filler element to produce cell
        element = 'Fe'
        unit_cell = bulk(element, structure, a = lattice_constant, cubic = True)
        supercell = unit_cell.repeat((s,s,s)) # Change supercell size here

        atom_list = []
        for atom, count in zip(atom_types, i):
            atom_list.extend([atom] * count)
        
        supercell.set_chemical_symbols(atom_list)
        write(filename, supercell, format = 'vasp', direct = True)
        shuffle_poscar(filename, permutation)
    return


In [3]:
# Example usage:
atom_types = ['Mo', 'V', 'Nb', 'Ti', 'Zr']
atom_count = 128
elements = len(atom_types)
minimum = 20

permutation = 0
structure = 'bcc'

# creating multiple combinations of different distributions, storing them as lists
combinations = calculate_combinations(atom_count, elements, minimum) 

# combinations can also be replaced with a list made by user:

# # Expected high pugh ratios - more ductile
# com1 = [60, 20, 8, 20, 20]
# com2 = [50, 15, 8, 20, 35]
# com3 = [65, 10, 5, 15, 33]

# # Expected low pugh ratios - more brittle
# com4 = [8, 20, 60, 20, 20]
# com5 = [8, 35, 50, 20, 15]
# com6 = [5, 33, 65, 15, 10]

# combinations = [com1, com2, com3, com4, com5, com6]

#gen_mass_poscar(atom_types, combinations, permutation, structure = 'bcc', folder = 'test')

## Generating Intermetallics from Materials Project API 

In [ ]:
from mp_api.client import MPRester
from jarvis.core.atoms import Atoms
import os
import pandas as pd

import itertools

elements = ['Al', 'Co', 'Cr', 'Cu', 'Fe', 'Mn', 'Ni', 'Ti', 'V', 'Mo']
# Generate all possible combinations
comb2 = [list(comb) for comb in itertools.combinations(elements, 2)]
comb3 = [list(comb) for comb in itertools.combinations(elements, 3)]

combinations = comb2 + comb3

for comb in combinations:
    print(comb)

len(combinations)


with MPRester("key-id-here") as mpr:
        # output folder to store all poscar files
        output = 'intermetallics_2and3'
        os.makedirs(output, exist_ok=True)
        data_list = []
        for comb in combinations:
            material = mpr.materials.summary.search(
                    elements=comb,
                    num_elements=len(comb),
                    fields=['material_id', 'structure', 'symmetry', 'shear_modulus', 'bulk_modulus', 'formula_pretty'])

            for compound in material:
                data = {
                     'material': compound.formula_pretty,
                     'material_id': compound.material_id,
                     'symmetry': compound.symmetry.crystal_system,
                     'shear_modulus': compound.shear_modulus['voigt'] if compound.shear_modulus else None,
                     'bulk_modulus': compound.bulk_modulus['voigt'] if compound.bulk_modulus else None
                }

                crystal = compound.symmetry.crystal_system
                structure = compound.structure.to_conventional()
                filename = f"{output}/{compound.formula_pretty}_{compound.material_id}_{crystal}.vasp"
                structure.to(fmt='poscar', filename = filename)
                data_list.append(data)
                
df = pd.DataFrame(data_list)
# df here contains material, meterial_id, symmetry, shear_modulus and bulk modulus